In [1]:
%pip install pyexpat
%pip install scikit-learn
%pip install tqdm
%pip install pandas
%pip install spacy
!python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pyexpat (from versions: none)
ERROR: No matching distribution found for pyexpat


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 653.6 kB/s eta 0:00:20
     --------------------------------------- 0.1/12.8 MB 544.7 kB/s eta 0:00:24
     --------------------------------------- 0.1/12.8 MB 544.7 kB/s eta 0:00:24
     --------------------------------------- 0.1/12.8 MB 357.2 kB/s eta 0:00:36
     --------------------------------------- 0.1/12.8 MB 357.2 kB/s eta 0:00:36
     --------------------------------------- 0.1/12.8 MB 327.7 kB/s eta 0:00:39
     ----------------------------

In [29]:
from sklearn.neighbors import KNeighborsClassifier

from tqdm import tqdm
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
import spacy

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [30]:
import json


def preTraitement():
    data_list = []

    with open('goodreads_reviews_spoiler.json', 'r') as fichier_json:
        lines = fichier_json.readlines()  
        for ligne in tqdm(lines, desc='Chargement des données'):
            try:
                objet_json = json.loads(ligne)
                data_list.append(objet_json)
            except json.JSONDecodeError:
                pass

    df = pd.json_normalize(data_list)

    df = df[df['has_spoiler'] == True]

    data_json = df.to_dict(orient='records')

    print(bcolors.OKBLUE + "Ecriture dans onlyspoilers.json..." + bcolors.ENDC)
    with open('onlyspoilers.json', 'w') as fichier_resultat:
        json.dump(data_json, fichier_resultat, indent=4)
    print(bcolors.OKGREEN + "OK" + bcolors.ENDC)

In [31]:
def spoilersFilter():
    # Charger les données depuis le fichier JSON
    print(bcolors.OKBLUE + "Chargement des données..." + bcolors.ENDC)
    df = pd.read_json('onlyspoilers.json')
    print(bcolors.OKGREEN + "OK" + bcolors.ENDC)

    # Filtrage des données
    print(bcolors.OKBLUE + "Filtrage..." + bcolors.ENDC)
    df = df.drop_duplicates(subset=['user_id', 'book_id'])
    df['full_text'] = df['review_sentences'].apply(lambda sentences: ' '.join([sentence[1] for sentence in sentences]))
    df['label'] = df['rating'].apply(lambda x: 1 if x >= 3 else 0)  # 1 pour favorable, 0 pour défavorable
    df = df[['rating', 'label', 'full_text']]
    df = df.dropna(subset=['rating', 'label', 'full_text'])
    print(bcolors.OKGREEN + "OK" + bcolors.ENDC)

    # Sélectionner 2500 lignes de chaque groupe 'label' (favorable ou défavorable)
    df_sampled = df.groupby('label', group_keys=False).apply(lambda x: x.sample(min(len(x), 2500)))

    # Écrire les données échantillonnées dans data.json
    print(bcolors.OKBLUE + "Ecriture dans data.json..." + bcolors.ENDC)
    data_json = df_sampled.to_dict(orient='records')
    with open('data.json', 'w') as fichier_resultat:
        json.dump(data_json, fichier_resultat, indent=4)
    print(bcolors.OKGREEN + "OK" + bcolors.ENDC)

In [32]:
#preTraitement()
spoilersFilter()

Chargement des données...
OK
Filtrage...
OK
Ecriture dans data.json...
OK


In [33]:
import string


print(bcolors.OKBLUE + "Lecture..." + bcolors.ENDC)
df = pd.read_json('data.json')
print(bcolors.OKGREEN + "OK" + bcolors.ENDC)

# Prétraitement avec spaCy
print(bcolors.OKBLUE + "Prétraitement avec spaCy..." + bcolors.ENDC)
punctuations = string.punctuation
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
print(bcolors.OKCYAN + "nlp traitement..." + bcolors.ENDC)
def preprocess(texts):
    return [' '.join([token.lemma_ for token in doc if not token.is_stop and token.lemma_ not in punctuations]) 
            for doc in tqdm(nlp.pipe(texts, batch_size=50), total=len(texts))]
df['processed_text'] = preprocess(df['full_text'].tolist())
print(bcolors.OKGREEN + "OK" + bcolors.ENDC)

# Vectorisation
print(bcolors.OKBLUE + "Vectorisation..." + bcolors.ENDC)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['processed_text'])
print(bcolors.OKGREEN + "OK" + bcolors.ENDC)

Lecture...
OK
Prétraitement avec spaCy...
nlp traitement...


100%|██████████| 5000/5000 [02:06<00:00, 39.67it/s]


OK
Vectorisation...
OK


In [40]:
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# KNN
print(bcolors.OKBLUE + "KNN..." + bcolors.ENDC)
knn_model = KNeighborsClassifier(n_jobs=-1)
knn_params = {'n_neighbors': [1, 3, 5, 7]}
knn_clf = GridSearchCV(knn_model, knn_params)
knn_clf.fit(X_train, y_train)
print(bcolors.OKGREEN + "OK" + bcolors.ENDC)


# Évaluation de KNN
print(bcolors.OKBLUE + "Évaluation de KNN..." + bcolors.ENDC)
y_pred_knn = knn_clf.predict(X_test)
print("KNN Classification Report")
print(classification_report(y_test, y_pred_knn, zero_division=1))
print(bcolors.OKGREEN + "OK" + bcolors.ENDC)

KNN...
OK
Évaluation de KNN...
KNN Classification Report
              precision    recall  f1-score   support

           0       0.58      0.37      0.46       485
           1       0.56      0.75      0.64       515

    accuracy                           0.57      1000
   macro avg       0.57      0.56      0.55      1000
weighted avg       0.57      0.57      0.55      1000

OK


In [41]:
y = df['rating']  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# KNN
print(bcolors.OKBLUE + "KNN..." + bcolors.ENDC)
knn_model = KNeighborsClassifier(n_jobs=-1)
knn_params = {'n_neighbors': [1, 3, 5, 7]}
knn_clf = GridSearchCV(knn_model, knn_params)
knn_clf.fit(X_train, y_train)
print(bcolors.OKGREEN + "OK" + bcolors.ENDC)


# Évaluation de KNN
print(bcolors.OKBLUE + "Évaluation de KNN..." + bcolors.ENDC)
y_pred_knn = knn_clf.predict(X_test)
print("KNN Classification Report")
print(classification_report(y_test, y_pred_knn, zero_division=1))
print(bcolors.OKGREEN + "OK" + bcolors.ENDC)

KNN...
OK
Évaluation de KNN...
KNN Classification Report
              precision    recall  f1-score   support

           0       0.13      0.30      0.18        57
           1       0.08      0.04      0.05       122
           2       0.39      0.45      0.42       321
           3       0.23      0.06      0.10       130
           4       0.25      0.31      0.28       227
           5       0.30      0.24      0.27       143

    accuracy                           0.28      1000
   macro avg       0.23      0.23      0.22      1000
weighted avg       0.27      0.28      0.26      1000

OK


In [39]:
from sklearn.ensemble import RandomForestClassifier

# RandomForest
print(bcolors.OKBLUE + "RandomForest..." + bcolors.ENDC)
rf_model = RandomForestClassifier(n_jobs=-1)
rf_params = {'n_estimators': [10, 50, 100], 'max_depth': [10, 20, 30]}
rf_clf = GridSearchCV(rf_model, rf_params)
rf_clf.fit(X_train, y_train)
print(bcolors.OKGREEN + "OK" + bcolors.ENDC)

# Évaluation de RandomForest
print(bcolors.OKBLUE + "Évaluation de RandomForest..." + bcolors.ENDC)
y_pred_rf = rf_clf.predict(X_test)
print("RandomForest Classification Report")
print(classification_report(y_test, y_pred_rf, zero_division=1))
print(bcolors.OKGREEN + "OK" + bcolors.ENDC)


RandomForest...
OK
Évaluation de RandomForest...
RandomForest Classification Report
              precision    recall  f1-score   support

           0       1.00      0.00      0.00        57
           1       0.50      0.01      0.02       122
           2       0.33      0.99      0.50       321
           3       1.00      0.00      0.00       130
           4       0.60      0.05      0.10       227
           5       0.53      0.06      0.11       143

    accuracy                           0.34      1000
   macro avg       0.66      0.19      0.12      1000
weighted avg       0.57      0.34      0.20      1000

OK
